In [8]:

import os
import json5
import pandas as pd

# Define the path to the JSON directory
json_directory_path = "/Users/maqa/Documents/ice challange/datasets/json"

# Verify available JSON files
print("Files detected in directory:")
for json_filename in os.listdir(json_directory_path):
    print("  -", json_filename)


Files detected in directory:
  - .DS_Store
  - montreal_woman.json
  - ISU STWT 1 Men 500m.json
  - seoul_man.json
  - seoul_woman.json
  - montreal_man.json


In [9]:
seoul_man_json_path = os.path.join(json_directory_path, "seoul_man.json")
seoul_woman_json_path = os.path.join(json_directory_path, "seoul_woman.json")

montreal_man_json_path = os.path.join(json_directory_path, "montreal_man.json")
montreal_woman_json_path = os.path.join(json_directory_path, "montreal_man.json")

In [10]:

def show_all_json_paths(file_path: str):
    """
    Print every possible JSON key path from a JSON file.
    Works even if the file starts or ends incorrectly (missing braces, stray commas, etc.).
    """

    def traverse_json_structure(node, path="root"):
        """Recursively traverse nested JSON and print full key paths."""
        if isinstance(node, dict):
            for key, value in node.items():
                new_path = f"{path}.{key}"
                print(new_path)
                traverse_json_structure(value, new_path)
        elif isinstance(node, list):
            for index, value in enumerate(node):
                new_path = f"{path}[{index}]"
                print(new_path)
                traverse_json_structure(value, new_path)

    # === Load JSON safely ===
    with open(file_path, "r", encoding="utf-8") as file:
        raw_content = file.read().strip()

    # Ensure proper JSON structure even if malformed
    if not raw_content.startswith("{"):
        raw_content = "{" + raw_content
    if not raw_content.endswith("}"):
        raw_content = raw_content + "}"

    try:
        json_data = json5.loads(raw_content)
    except Exception as error:
        print(f"Error parsing JSON: {error}")
        return

    # Traverse and print all key paths
    traverse_json_structure(json_data)


In [11]:
show_all_json_paths(seoul_man_json_path)


root.Data
root.Data.Id
root.Data.Name
root.Data.EventId
root.Data.EventName
root.Data.SportCode
root.Data.DisplayOrder
root.Data.DisciplineName
root.Data.DisciplineDistance
root.Data.Start
root.Data.Start.Date
root.Data.Start.Year
root.Data.Start.Month
root.Data.Start.Day
root.Data.Start.Hour
root.Data.Start.Minute
root.Data.Start.TimeZone
root.Data.Gender
root.Data.Status
root.Data.Competitors
root.Data.Competitors[0]
root.Data.Competitors[0].Id
root.Data.Competitors[0].Competitor
root.Data.Competitors[0].Competitor.Person
root.Data.Competitors[0].Competitor.Person.Id
root.Data.Competitors[0].Competitor.Person.FirstName
root.Data.Competitors[0].Competitor.Person.LastName
root.Data.Competitors[0].Competitor.Person.Gender
root.Data.Competitors[0].Competitor.Person.OrganizationCode
root.Data.Competitors[0].Competitor.Person.DateOfBirth
root.Data.Competitors[0].Competitor.Person.DateOfBirth.Date
root.Data.Competitors[0].Competitor.Person.DateOfBirth.Year
root.Data.Competitors[0].Competito

we need following dataframes :
events_df.csv
competitors_df.csv
rounds_df.csv
heats_df.csv
heat_competitors_df.csv
laps_df.csv


In [12]:

# === Load JSON ===
with open(seoul_man_json_path, "r", encoding="utf-8") as json_file:
    raw_content = json_file.read().strip()


# ensure valid JSON braces
if not raw_content.startswith("{"):
    raw_content = "{" + raw_content
if not raw_content.endswith("}"):
    raw_content = raw_content + "}"

data = json5.loads(raw_content)

# move into "Data" node
event_data = data.get("Data", {})

# inspect top-level keys
print("Top-level keys in Data:", list(event_data.keys()))

Top-level keys in Data: ['Id', 'Name', 'EventId', 'EventName', 'SportCode', 'DisplayOrder', 'DisciplineName', 'DisciplineDistance', 'Start', 'Gender', 'Status', 'Competitors', 'Rounds']


In [13]:
event_record = {
    "event_id": event_data.get("Id"),
    "event_name": event_data.get("EventName"),
    "discipline_name": event_data.get("DisciplineName"),
    "discipline_distance": event_data.get("DisciplineDistance"),
    "sport_code": event_data.get("SportCode"),
    "gender": event_data.get("Gender"),
    "display_order": event_data.get("DisplayOrder"),
    "status": event_data.get("Status"),
    "start_date": event_data.get("Start", {}).get("Date"),
    "start_year": event_data.get("Start", {}).get("Year"),
    "start_month": event_data.get("Start", {}).get("Month"),
    "start_day": event_data.get("Start", {}).get("Day"),
    "start_hour": event_data.get("Start", {}).get("Hour"),
    "start_minute": event_data.get("Start", {}).get("Minute"),
    "time_zone": event_data.get("Start", {}).get("TimeZone"),
}

events_df = pd.DataFrame([event_record])


seoul_man.json is one single event

In [14]:
competitors_list = event_data.get("Competitors", [])

# === Extract relevant fields ===
competitor_records = []

for competitor_entry in competitors_list:
    competitor_info = competitor_entry.get("Competitor", {})
    person_info = competitor_info.get("Person", {})

    record = {
        "competitor_id": competitor_entry.get("Id"),
        "bib_number": competitor_entry.get("BibNumber"),
        "final_rank": competitor_entry.get("FinalRank"),
        "final_rank_index": competitor_entry.get("FinalRankIndex"),
        "final_result": competitor_entry.get("FinalResult"),
        "final_position": competitor_entry.get("FinalPosition"),
        "result_status": competitor_entry.get("ResultStatus"),
        "preceding_positions": competitor_entry.get("PrecedingPositions"),

        # Person-level fields
        "person_id": person_info.get("Id"),
        "first_name": person_info.get("FirstName"),
        "last_name": person_info.get("LastName"),
        "gender": person_info.get("Gender"),
        "organization_code": person_info.get("OrganizationCode"),

        # National info
        "started_for_nf_code": competitor_info.get("StartedForNfCode"),
        "started_for_nf_name": competitor_info.get("StartedForNfName"),
        "started_for_nf_country_name": competitor_info.get("StartedForNfCountryName"),

        # Birth details
        "birth_date": person_info.get("DateOfBirth", {}).get("Date"),
        "birth_year": person_info.get("DateOfBirth", {}).get("Year"),
        "birth_month": person_info.get("DateOfBirth", {}).get("Month"),
        "birth_day": person_info.get("DateOfBirth", {}).get("Day"),
    }

    competitor_records.append(record)

# === Create DataFrame ===
competitors_df = pd.DataFrame(competitor_records)
print(f"Number of competitors: {len(competitors_df)}")
print(competitors_df.head())


Number of competitors: 74
                          competitor_id  bib_number  final_rank  \
0  f75b0c0f-107d-4474-bb47-45b2926ff298           2           1   
1  c4b9df64-f59a-4f18-b837-0de23875f136           7           2   
2  013f54e5-d12a-4970-84bb-e2e40a383950          19           3   
3  a15aaec4-779d-4ca4-916a-6589b7678c2f          46           4   
4  2560bda6-2a9e-4b0c-8adf-a91b273d71b2          42           5   

   final_rank_index final_result final_position result_status  \
0                 1       40.745           None          None   
1                 2       40.920           None          None   
2                 3       40.856           None          None   
3                 4       40.857           None          None   
4                 5       40.822           None          None   

  preceding_positions                             person_id first_name  \
0                None  0508a236-47d3-48e1-9de9-3168455e3f67     Steven   
1                None  aecaf5f1-

In [15]:
# === Step 3: Extract Rounds ===

rounds_list = event_data.get("Rounds", [])
round_records = []

for round_entry in rounds_list:
    start_info = round_entry.get("Start", {})

    record = {
        "round_name": round_entry.get("Name"),
        "display_order": round_entry.get("DisplayOrder"),
        "state": round_entry.get("State"),
        "start_date": start_info.get("Date"),
        "start_year": start_info.get("Year"),
        "start_month": start_info.get("Month"),
        "start_day": start_info.get("Day"),
        "start_hour": start_info.get("Hour"),
        "start_minute": start_info.get("Minute"),
        "time_zone": start_info.get("TimeZone"),
        "num_heats": len(round_entry.get("Heats", []))
    }

    round_records.append(record)

rounds_df = pd.DataFrame(round_records)
print(f"Number of rounds: {len(rounds_df)}")
print(rounds_df.head())


Number of rounds: 7
           round_name  display_order state                 start_date  \
0       Preliminaries              1  None  2024-12-13T10:13:00+00:00   
1               Heats              2  None  2024-12-13T14:58:00+00:00   
2  Rep. Quarterfinals              3  None  2024-12-14T11:38:00+00:00   
3     Rep. Semifinals              4  None  2024-12-14T12:17:00+00:00   
4       Quarterfinals              5  None  2024-12-14T15:22:00+00:00   

   start_year  start_month  start_day  start_hour  start_minute time_zone  \
0        2024           12         13          10            13   Etc/UTC   
1        2024           12         13          14            58   Etc/UTC   
2        2024           12         14          11            38   Etc/UTC   
3        2024           12         14          12            17   Etc/UTC   
4        2024           12         14          15            22   Etc/UTC   

   num_heats  
0         11  
1          8  
2          8  
3          2  
4  

In [16]:
# === Step 4: Extract Heats from all Rounds ===

heat_records = []

for round_entry in event_data.get("Rounds", []):
    round_name = round_entry.get("Name")
    heats = round_entry.get("Heats", [])
    
    for heat in heats:
        start_info = heat.get("Start", {})
        
        record = {
            "round_name": round_name,
            "heat_id": heat.get("Id"),
            "heat_name": heat.get("Name"),
            "display_order": heat.get("DisplayOrder"),
            "result_status": heat.get("ResultStatus"),
            "status": heat.get("Status"),
            "start_date": start_info.get("Date"),
            "start_year": start_info.get("Year"),
            "start_month": start_info.get("Month"),
            "start_day": start_info.get("Day"),
            "start_hour": start_info.get("Hour"),
            "start_minute": start_info.get("Minute"),
            "time_zone": start_info.get("TimeZone"),
            "photo_finish_url": heat.get("PhotoFinishUrl"),
            "num_competitors": len(heat.get("Competitors", [])),
        }
        
        heat_records.append(record)

heats_df = pd.DataFrame(heat_records)
print(f"Number of heats: {len(heats_df)}")
print(heats_df.head())


Number of heats: 37
      round_name                               heat_id heat_name  \
0  Preliminaries  c7094c79-ab1a-427c-8c72-fea4f13ddbdb    Heat 1   
1  Preliminaries  af201b61-57e2-4cb7-a185-6bfc53d8f0cd    Heat 2   
2  Preliminaries  4b767742-7b21-45bf-9bc7-dac58bf37958    Heat 3   
3  Preliminaries  8f94c91d-93f1-42af-8e86-ee44622e6970    Heat 4   
4  Preliminaries  4841c03d-0409-47e4-ba40-0451b0624192    Heat 5   

   display_order result_status    status                 start_date  \
0              1      Official  Finished  2024-12-13T10:23:11+00:00   
1              2      Official  Finished  2024-12-13T10:25:34+00:00   
2              3      Official  Finished  2024-12-13T10:28:03+00:00   
3              4      Official  Finished  2024-12-13T10:32:32+00:00   
4              5      Official  Finished  2024-12-13T10:33:00+00:00   

   start_year  start_month  start_day  start_hour  start_minute time_zone  \
0        2024           12         13          10            23   E

In [17]:
# === Step 5: Extract All Competitors (Global List) ===
competitor_records = []
for comp in event_data.get("Competitors", []):
    person = comp.get("Competitor", {}).get("Person", {})
    record = {
        "competition_competitor_id": comp.get("Id"),
        "first_name": person.get("FirstName"),
        "last_name": person.get("LastName"),
        "gender": person.get("Gender"),
        "organization_code": person.get("OrganizationCode"),
        "started_for_nf_country_name": comp.get("Competitor", {}).get("StartedForNfCountryName"),
        "started_for_nf_name": comp.get("Competitor", {}).get("StartedForNfName"),
        "started_for_nf_code": comp.get("Competitor", {}).get("StartedForNfCode"),
        "date_of_birth": person.get("DateOfBirth", {}).get("Date"),
    }
    competitor_records.append(record)

competitors_df = pd.DataFrame(competitor_records)
print(f"Number of competitors: {len(competitors_df)}")
print(competitors_df.head(5))

Number of competitors: 74
              competition_competitor_id first_name      last_name gender  \
0  f75b0c0f-107d-4474-bb47-45b2926ff298     Steven         DUBOIS      M   
1  c4b9df64-f59a-4f18-b837-0de23875f136     Jordan  PIERRE-GILLES      M   
2  013f54e5-d12a-4970-84bb-e2e40a383950    Roberts      KRUZBERGS      M   
3  a15aaec4-779d-4ca4-916a-6589b7678c2f      Shogo         MIYATA      M   
4  2560bda6-2a9e-4b0c-8adf-a91b273d71b2    Brandon            KIM      M   

  organization_code started_for_nf_country_name          started_for_nf_name  \
0               CAN                      Canada         Speed Skating Canada   
1               CAN                      Canada         Speed Skating Canada   
2               LAT                      Latvia  Latvian Skating Association   
3               JPN                       Japan     Japan Skating Federation   
4               USA    United States of America            U.S. Speedskating   

  started_for_nf_code              d

In [27]:
# === Step 6: Extract Competitors within Each Heat (Performances) ===
heat_competitor_records = []

for round_entry in event_data.get("Rounds", []):
    round_name = round_entry.get("Name")
    for heat in round_entry.get("Heats", []):
        heat_id = heat.get("Id")
        heat_name = heat.get("Name")

        for competitor in heat.get("Competitors", []):
            c = competitor.get("Competitor", {})
            person = c.get("Person", {})

            record = {
                "round_name": round_name,
                "heat_id": heat_id,
                "heat_name": heat_name,

                # Try multiple levels for competition_competitor_id
                "competition_competitor_id": (
                    competitor.get("CompetitionCompetitorId")
                    or competitor.get("Id")
                    or c.get("Id")
                    or person.get("Id")
                ),

                "bib_number": competitor.get("BibNumber"),
                "starting_position": competitor.get("StartingPosition"),
                "final_rank": competitor.get("FinalRank"),
                "final_result": competitor.get("FinalResult"),
                "final_points": competitor.get("FinalPoints"),
                "result_status": competitor.get("ResultStatus"),
                "qualification_code": competitor.get("QualificationCode"),
                "record_flag": competitor.get("RecordFlag"),
                "num_laps": len(competitor.get("Laps", [])),
            }

            heat_competitor_records.append(record)

heat_competitors_df = pd.DataFrame(heat_competitor_records)
print(f"✅ Number of heat-competitor records: {len(heat_competitors_df)}")
print(heat_competitors_df[["competition_competitor_id", "final_rank", "final_result"]].head(10))


✅ Number of heat-competitor records: 185
              competition_competitor_id  final_rank final_result
0  a9d7de7a-d561-43e7-9d2b-a25f5b720ea8         1.0       42.162
1  6966756c-063a-4dad-8481-de3a5060fba8         2.0       42.232
2  aec5d4c4-5b9a-49a6-9803-d76affd5bdae         3.0       42.356
3  dc80c9c3-094f-4762-a72d-5dc47a1b9d47         4.0       42.379
4  69ff363e-e62c-408d-934a-8315964a100d         5.0       42.650
5  29f872c9-cead-450c-9997-3b6d6dacc1e8         1.0       41.718
6  17cf500c-d597-4fd2-ac27-62e8aabe6d11         2.0       41.909
7  996efec1-974c-4d9d-8e24-17babe3ddf17         3.0       41.928
8  e8c2bb25-db3d-416c-a9d4-49f1721c7e1e         4.0       42.714
9  684d77eb-e2b4-4b0f-b5aa-7a32a1cc99ac         5.0       43.557


In [28]:
# === Step 7: Extract all Laps from all Competitors in all Heats ===

lap_records = []

for round_entry in event_data.get("Rounds", []):
    round_name = round_entry.get("Name")
    
    for heat in round_entry.get("Heats", []):
        heat_id = heat.get("Id")
        heat_name = heat.get("Name")
        
        for competitor in heat.get("Competitors", []):
            competition_competitor_id = competitor.get("CompetitionCompetitorId")
            bib_number = competitor.get("BibNumber")
            
            laps = competitor.get("Laps", [])
            for lap in laps:
                record = {
                    "round_name": round_name,
                    "heat_id": heat_id,
                    "heat_name": heat_name,
                    "competition_competitor_id": competition_competitor_id,
                    "bib_number": bib_number,
                    "lap_number": lap.get("LapNumber"),
                    "rank": lap.get("Rank"),
                    "lap_time": lap.get("LapTime"),
                    "total_time": lap.get("Time"),
                    "result_diff": lap.get("ResultDiff"),
                }
                lap_records.append(record)

laps_df = pd.DataFrame(lap_records)
print(f"Number of lap records: {len(laps_df)}")
print(laps_df.head(15))


Number of lap records: 853
       round_name                               heat_id heat_name  \
0   Preliminaries  c7094c79-ab1a-427c-8c72-fea4f13ddbdb    Heat 1   
1   Preliminaries  c7094c79-ab1a-427c-8c72-fea4f13ddbdb    Heat 1   
2   Preliminaries  c7094c79-ab1a-427c-8c72-fea4f13ddbdb    Heat 1   
3   Preliminaries  c7094c79-ab1a-427c-8c72-fea4f13ddbdb    Heat 1   
4   Preliminaries  c7094c79-ab1a-427c-8c72-fea4f13ddbdb    Heat 1   
5   Preliminaries  c7094c79-ab1a-427c-8c72-fea4f13ddbdb    Heat 1   
6   Preliminaries  c7094c79-ab1a-427c-8c72-fea4f13ddbdb    Heat 1   
7   Preliminaries  c7094c79-ab1a-427c-8c72-fea4f13ddbdb    Heat 1   
8   Preliminaries  c7094c79-ab1a-427c-8c72-fea4f13ddbdb    Heat 1   
9   Preliminaries  c7094c79-ab1a-427c-8c72-fea4f13ddbdb    Heat 1   
10  Preliminaries  c7094c79-ab1a-427c-8c72-fea4f13ddbdb    Heat 1   
11  Preliminaries  c7094c79-ab1a-427c-8c72-fea4f13ddbdb    Heat 1   
12  Preliminaries  c7094c79-ab1a-427c-8c72-fea4f13ddbdb    Heat 1   
13  Pre

In [29]:
print("=== DataFrames currently loaded ===")
for name, value in dict(globals()).items():
    if isinstance(value, pd.DataFrame):
        print(f"{name:35s}  shape={value.shape} ")


=== DataFrames currently loaded ===
events_df                            shape=(1, 16) 
competitors_df                       shape=(74, 10) 
rounds_df                            shape=(7, 12) 
heats_df                             shape=(37, 16) 
heat_competitors_df                  shape=(185, 13) 
laps_df                              shape=(853, 10) 
df                                   shape=(853, 11) 


In [30]:
# Add json_source column to all core DataFrames
for df in [events_df, competitors_df, rounds_df, heats_df, heat_competitors_df, laps_df]:
    df["json_source"] = "seoul_man"


In [31]:
# Directory to store CSV files
export_dir = "processed_datasets/seoul_man"
os.makedirs(export_dir, exist_ok=True)

# Core DataFrames to export
dataframes = {
    "events": events_df,
    "competitors": competitors_df,
    "rounds": rounds_df,
    "heats": heats_df,
    "heat_competitors": heat_competitors_df,
    "laps": laps_df
}

# Export each DataFrame to CSV
for name, df in dataframes.items():
    csv_path = os.path.join(export_dir, f"{name}.csv")
    df.to_csv(csv_path, index=False)
    print(f"Saved {name}.csv  →  {csv_path}")

print("\nAll CSVs saved successfully in:", os.path.abspath(export_dir))


Saved events.csv  →  processed_datasets/seoul_man/events.csv
Saved competitors.csv  →  processed_datasets/seoul_man/competitors.csv
Saved rounds.csv  →  processed_datasets/seoul_man/rounds.csv
Saved heats.csv  →  processed_datasets/seoul_man/heats.csv
Saved heat_competitors.csv  →  processed_datasets/seoul_man/heat_competitors.csv
Saved laps.csv  →  processed_datasets/seoul_man/laps.csv

All CSVs saved successfully in: /Users/maqa/Documents/ice challange/noteboooks/processed_datasets/seoul_man


In [26]:
print(heat_competitors_df.head(2))

      round_name                               heat_id heat_name  \
0  Preliminaries  c7094c79-ab1a-427c-8c72-fea4f13ddbdb    Heat 1   
1  Preliminaries  c7094c79-ab1a-427c-8c72-fea4f13ddbdb    Heat 1   

  competition_competitor_id  bib_number  starting_position  final_rank  \
0                      None          31                  1         1.0   
1                      None         145                  2         2.0   

  final_result final_points result_status qualification_code record_flag  \
0       42.162         None          None                  Q        None   
1       42.232         None          None                  q        None   

   num_laps json_source  
0         5   seoul_man  
1         5   seoul_man  
